In [ ]:
from scipy.stats import multivariate_normal
import cv2
from scipy import io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.ndimage import label,map_coordinates,convolve1d, rank_filter
from numpy import unravel_index
import math
import glob
import base64
from numpy.linalg import norm
import os
import sys
%matplotlib inline

global ct
ct=0

In [ ]:
#step0 -> read images
I11= cv2.imread("part3/1.jpg")
I12= cv2.imread("part3/2.jpg")
I13= cv2.imread("part3/3.jpg")
I14= cv2.imread("part3/4.jpg")
I15= cv2.imread("part3/5.jpg")
I16= cv2.imread("part3/6.jpg")

I11 = cv2.cvtColor(I11, cv2.COLOR_BGR2RGB)
I12 = cv2.cvtColor(I12, cv2.COLOR_BGR2RGB)
I13 = cv2.cvtColor(I13, cv2.COLOR_BGR2RGB)
I14 = cv2.cvtColor(I14, cv2.COLOR_BGR2RGB)
I15 = cv2.cvtColor(I15, cv2.COLOR_BGR2RGB)
I16 = cv2.cvtColor(I16, cv2.COLOR_BGR2RGB)


fig1, ax1 = plt.subplots(1, 3, figsize = (10,30))
ax1[0].imshow(I11)
ax1[1].imshow(I12)
ax1[2].imshow(I13)
ax1[0].set_title("Image 1")
ax1[1].set_title("Image 2")
ax1[2].set_title("Image 3")
title='part3-results/'
name=str(ct)
plt.savefig(title+name,facecolor='w')
ct=ct+1

fig2, ax2 = plt.subplots(1, 3, figsize = (10,30))
ax2[0].imshow(I14)
ax2[1].imshow(I15)
ax2[2].imshow(I16)
ax2[0].set_title("Image 4")
ax2[1].set_title("Image 5")
ax2[2].set_title("Image 6")

print(type(I11[0][0][0]))

title='part3-results/'
name=str(ct)
plt.savefig(title+name,facecolor='w')
ct=ct+1

In [ ]:
def fix_image(img, t, dims):
    new_image_map = {}
    minx, miny = img.shape[0], img.shape[1]
    maxx, maxy = 0, 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            xy = np.array([i, j, 1], np.float64)
            uv = np.matmul(t, xy)
            uv = uv / uv[2]
            minx = min(minx, uv[0])
            maxx = max(maxx, uv[0])
            miny = min(miny, uv[1])
            maxy = max(maxy, uv[1])

            new_image_map[int(uv[0]), int(uv[1])] = (i, j)

    minx, miny = int(minx), int(miny)
    maxx, maxy = int(maxx), int(maxy)
    final_img = np.zeros((maxx - minx + 1, maxy - miny + 1),dtype=np.int) \
        if len(img.shape) == 2 else np.zeros((maxx - minx + 1, maxy - miny + 1, img.shape[2]),dtype=np.int)

    for k, v in new_image_map.items():
        final_img[k[0] - minx, k[1] - miny] = img[v]

    return final_img

In [ ]:
def perspectiveTransformMan(perspectiveMatrix, sourcePoints):
    '''
    perspectiveMatrix as above
    sourcePoints has shape (n,2)
    '''

    # first we extend source points by a column of 1
    # augment has shape (n,1)
    
    augment = np.ones((sourcePoints.shape[0],1))
    # projective_corners is a 3xn matrix with last row all 1
    # note that we transpose the concatenation
    projective_corners = np.concatenate( (sourcePoints, augment), axis=1).T

    # projective_points has shape 3xn
    projective_points = perspectiveMatrix.dot(projective_corners)

    # obtain the target_points by dividing the projective_points 
    # by its last row (where it is non-zero)
    # target_points has shape (3,n).
    target_points = np.true_divide(projective_points, projective_points[-1])

    # so we want return points in row form
    return target_points[:2].T




In [ ]:
def warpImages(img1, img2, H): #img1 statheri kai img2 warped
    
    global ct

    rows1, cols1 = img1.shape[:2]
    rows2, cols2 = img2.shape[:2]

    list_of_points_1 = np.float32([[0,0], [0, rows1],[cols1, rows1], [cols1, 0]]) #.reshape(-1, 1, 2)
    temp_points = np.float32([[0,0], [0,rows2], [cols2,rows2], [cols2,0]]) #.reshape(-1,1,2)

    # When we have established a homography we need to warp perspective
    # Change field of view
    list_of_points_2 = perspectiveTransformMan(H,temp_points) #.reshape(-1, 1, 2)
    
    list_of_points_3 = perspectiveTransformMan(np.linalg.inv(H),list_of_points_2)
    
    
    [x_min_2, y_min_2] = np.int32(list_of_points_2.min(axis=0).ravel() - 0.5)
    [x_max_2, y_max_2] = np.int32(list_of_points_2.max(axis=0).ravel() + 0.5)

    list_of_points = np.concatenate((list_of_points_1,list_of_points_2), axis=0)

    [x_min, y_min] = np.int32(list_of_points.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(list_of_points.max(axis=0).ravel() + 0.5)
  
    translation_dist = [x_min,y_min]
    katw_deksia = [x_max,y_max]
    print("Top Left Coords: ", translation_dist)
    
    output_img = np.zeros((y_max - y_min, x_max - x_min, 3),dtype=np.uint8)
    
    
    
    for i in range(0,rows2):
        for j in range(0,cols2):
            temp=np.float32([[j,i]])
            points = perspectiveTransformMan(H,temp)
            points = np.round(points)
            output_img[int(points[0][1]+abs(translation_dist[1])),int(points[0][0]+abs(translation_dist[0]))]=img2[i,j]
          
     
    #hsv = cv2.cvtColor(output_img, cv2.COLOR_RGB2HSV)
    #hsv[:,:,2] += 10
    #output_img=cv2.cvtColor(hsv,cv2.COLOR_HSV2RGB)
    
    output_img=output_img.astype(np.uint8)
    plt.imshow(output_img)
    plt.title("Image 1 warped")
    title='part3-results/'
    name=str(ct)
    plt.savefig(title+name)
    ct=ct+1
    plt.show()
    
    
            
            
            
    
    for i in range(0,rows1):
        for j in range(0,cols1):
            mid_sum=img1[i,j][0]+img1[i,j][1]+img1[i,j][2]
            if (mid_sum!=0): output_img[i+abs(translation_dist[1]),j+abs(translation_dist[0])]=img1[i,j]
            
    #hsv = cv2.cvtColor(output_img, cv2.COLOR_RGB2HSV)
    #hsv[:,:,2] += 10
    #output_img=cv2.cvtColor(hsv,cv2.COLOR_HSV2RGB)
                
    
    output_img=output_img.astype(np.uint8)
    plt.imshow(output_img)
    plt.title("Stitched image")
    title='part3-results/'
    name=str(ct)
    plt.savefig(title+name)
    ct=ct+1
    plt.show()
    
    return(output_img)

In [ ]:
def image_stiching(I1,I2):
    
    global ct

    #step 1
    sift = cv2.xfeatures2d.SIFT_create()
    key_1,descr_1 = sift.detectAndCompute(I1,None)
    feats_1 = cv2.drawKeypoints(I1,key_1,0)
    key_2,descr_2 = sift.detectAndCompute(I2,None)
    feats_2 = cv2.drawKeypoints(I2,key_2,0)
    
    fig, ax = plt.subplots(1, 2, figsize = (10,30))
    ax[0].imshow(feats_1)
    ax[1].imshow(feats_2)
    ax[0].set_title("Features Image 1")
    ax[1].set_title("Features Image 2")
    
    title='part3-results/'
    name=str(ct)
    plt.savefig(title+name)
    ct=ct+1
    
    plt.show()
    
    I1g=I1
    I2g=I2
    
    #step_2

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descr_1,descr_2, k=2)

    I12 = cv2.drawMatchesKnn(I1g,key_1,I2g,key_2,matches,None,flags=2)

    plt.imshow(I12)
    plt.title("Matching with Brute Force Matcher")
    title='part3-results/'
    name=str(ct)
    plt.savefig(title+name)
    ct=ct+1
    plt.show()
    
    #step_3
    thres=0.8
    good = [[m] for m, n in matches if m.distance < thres*n.distance]
    good_not_list=[m for m, n in matches if m.distance < thres*n.distance]
    
    print("Features before Lowe Threshold: ", np.shape(matches)[0])
    print("Features after Lowe Threshold: ", np.shape(good_not_list)[0])

    I12_lowe = cv2.drawMatches(I1g,key_1,I2g,key_2,good_not_list,None,flags=2)

    plt.imshow(I12_lowe)
    plt.title("Matching after Lowe Threshold")
    title='part3-results/'
    name=str(ct)
    plt.savefig(title+name)
    ct=ct+1
    plt.show()
    
    #step_4
    query_pts = np.float32([key_1[m.queryIdx] 
                    .pt for m in good_not_list]).reshape(-1, 1, 2) 

    train_pts = np.float32([key_2[m.trainIdx] 
                    .pt for m in good_not_list]).reshape(-1, 1, 2)

    HBA, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC,4.0)

    inv_HBA=np.linalg.inv(HBA)
    print("The Homography matrix is:", HBA)
    
    #step 5+6

    final_final = warpImages(I2g,I1g,HBA)
    
    return final_final


In [ ]:
I12w=np.array(image_stiching(I11,I12))

In [ ]:
I123w=np.array(image_stiching(I13,I12w)) #warped,statheri
print(np.shape(I123w))


In [ ]:
#I1234w=np.array(image_stiching(I14,I123w)) #warped,statheri
#print(np.shape(I1234w))

In [ ]:
I45w=np.array(image_stiching(I14,I15)) #warped,statheri
print(np.shape(I45w))

In [ ]:
I456w=np.array(image_stiching(I16,I45w)) #warped,statheri
print(np.shape(I456w))

In [ ]:
Iol=np.array(image_stiching(I123w,I456w))
print(np.shape(Iol))

